In [ ]:
%load_ext kamu

In [ ]:
%import_dataset ca.vancouver.data.property-information.property-parcel-polygons --alias lots

In [ ]:
print(lots.count())
lots.printSchema()
lots.show(3)

In [ ]:
%import_dataset ca.vancouver.data.property-tax-report --alias tax

In [ ]:
print(tax.count())
tax.printSchema()
tax.show(3)

In [ ]:
%%sql
CREATE OR REPLACE TEMP VIEW lots_tax AS (
SELECT
    l.tax_coord,
    l.geometry, 
    t.current_land_value, 
    t.current_improvement_value
FROM lots as l
LEFT JOIN tax as t
ON l.tax_coord = t.land_coordinate
WHERE t.tax_assessment_year = 2018
)

In [ ]:
%%sql -o df -n 1000000
SELECT
    tax_coord,
    ST_AsText(geometry) as geometry,
--    current_land_value + current_improvement_value as current_total_value
    RAND() * 5000000 as current_total_value
--FROM lots_tax
FROM lots

In [ ]:
%%local
import shapely.wkt
import geojson

def df_to_geojson(df, geom='geometry', props=None):
    if props is None:
        props = [
            c for c in df.columns
            if c != geom
        ]

    return geojson.FeatureCollection([
        geojson.Feature(
            geometry=shapely.wkt.loads(r[geom]),
            properties={p: r[p] for p in props}
        )
        for _, r in df.iterrows()
    ])

In [ ]:
%%local
gj = df_to_geojson(df)

In [ ]:
%%local
import os
from mapboxgl.viz import *
from mapboxgl.utils import *

# Must be a public token, starting with `pk`
token = os.getenv('MAPBOX_ACCESS_TOKEN')

In [ ]:
%%local
viz = ChoroplethViz(
    gj,
    style='mapbox://styles/mapbox/dark-v10',
    center=(-123.1207, 49.2827),
    zoom=10,
    access_token=token,
    color_property='current_total_value',
    color_stops=create_color_stops([750000, 1000000, 2000000, 3000000, 5000000], colors='YlOrRd'),
    color_default='rgb(158,202,195)',
    line_stroke='solid',
    line_width=0.1,
    line_color='rgb(128,0,38)',
    opacity=0.8,
    legend_layout='horizontal',
    legend_key_shape='bar',
    legend_key_borders_on=False)

viz.show()